In [1]:
import sys
sys.path
sys.path.append("/home/haoqi.whq/llm-inference/LoRA")

In [2]:
from src.model import GPT2Config, GPT2LMModel
import torch
from loralib import PruneLayer
import loralib as lora
import copy
import importlib
importlib.reload(lora)


<module 'loralib' from '/home/haoqi.whq/llm-inference/LoRA/loralib/__init__.py'>

In [5]:
state_dict = torch.load("./pretrained_checkpoints/gpt2-medium-pytorch_model.bin")
print(state_dict)
def split_attention_weights(state_dict):
    state_dict_tmp = copy.deepcopy(state_dict)
    attn_lists = ["q_attn", "k_attn", "v_attn"]
    for n, p in state_dict.items():
        if "c_attn" in n:
            module, name = n.split("c_attn.")
            dim = 0 if name == "bias" else 1
            split_size = p.shape[0] if name == "weight" else p.shape[0] // 3
            params = p.split(split_size, dim=dim)
            for i, attn in enumerate(attn_lists):
                print(module+attn+"."+name)
                state_dict_tmp[module+attn+"."+name] = params[i]
    return state_dict_tmp

# state_dict_tmp = split_attention_weights(state_dict)
# print(state_dict_tmp["h.0.attn.q_attn.bias"].shape)

OrderedDict([('wte.weight', tensor([[-0.0115,  0.0031, -0.0073,  ..., -0.0526, -0.1757,  0.0257],
        [-0.0086,  0.0636, -0.0182,  ..., -0.0136, -0.1215,  0.0535],
        [ 0.0585,  0.0689,  0.0262,  ..., -0.1006, -0.1979, -0.0039],
        ...,
        [ 0.0016, -0.0441, -0.0517,  ..., -0.1008, -0.0087,  0.0264],
        [-0.1437, -0.0463, -0.0065,  ...,  0.0746, -0.0472, -0.0383],
        [ 0.0207, -0.0133, -0.0259,  ...,  0.0389, -0.0023,  0.0011]])), ('wpe.weight', tensor([[-0.0372, -0.0034, -0.0224,  ..., -0.0027,  0.0052, -0.0030],
        [ 0.0373, -0.0179, -0.0144,  ...,  0.0232, -0.0038,  0.0070],
        [-0.0017, -0.0226,  0.0179,  ...,  0.0196,  0.0221, -0.0140],
        ...,
        [ 0.0005, -0.0069,  0.0017,  ...,  0.0020,  0.0004,  0.0159],
        [-0.0032, -0.0033,  0.0011,  ...,  0.0036, -0.0045,  0.0179],
        [-0.0064,  0.0079, -0.0010,  ..., -0.0040, -0.0011, -0.0015]])), ('h.0.ln_1.weight', tensor([0.0613, 0.0561, 0.0508,  ..., 0.0560, 0.0389, 0.0597])), 

In [3]:
# medium GPT2
config = GPT2Config(
    n_embd=1024,
    n_layer=24,
    n_head=16,
    lora_attn_dim=4,
    lora_attn_alpha=32,
    lora_dropout=0.1,
    enable_mlp=True,
    enable_wo=True,
    enable_wq=True,
    enable_wk=False,
    enable_wv=True,
)

lm_net = GPT2LMModel(config)
# print(lm_net.state_dict())
# transformer.h.0.attn.q_atten.lora_B
# lm_head.decoder.weight
print("loading model pretrained weight.")
lm_net.load_lora_weight("./pretrained_checkpoints/gpt2-medium-pytorch_model.bin", torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.lora.1000.pt"))


O Attention using LoRA: PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
)
MLP using LoRA: PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
), PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
)
loading model pretrained weight.
{'h.0.attn.q_atten.lora_A': tensor([[-0.0248, -0.0026, -0.0073,  ...,  0.0132, -0.0211, -0.0391],
        [-0.0165,  0.0131,  0.0005,  ..., -0.0228, -0.0016,  0.0099],
        [-0.0203,  0.0291,  0.0184,  ...,  0.0062,  0.0032,  0.0107],
        [-0.0107,  0.0296,  0.0310,  ..., -0.0355, -0.0059,  0.0569]],
       device='cuda:0'), 'h.0.attn.q_atten.lora_B': tensor([[-0.0049, -0.0103, -0.0007,  0.0157],
        [ 0.0182, -0.0070, -0.0061, -0.0143],
        [ 0.0141, -0.0081, -0.0054, -0.0098],
        ...,
        [-0.0125, -0.0010, -0.0121, -0.0121],
        [-0.0234, -0.0078,  0.0068,  0.0076],
        [-0.0099,  0.0016,  0.0041,  0.0041]], device='cuda:0'), 'h.0.attn.q_atten.lora_scaling': tensor(7.8867, device='cud

In [ ]:
lora.mark_only_lora_as_trainable(lm_net)
for m in lm_net.modules():
    if isinstance(m, PruneLayer) and hasattr(m, "lora_scaling"):
        print(m.lora_scaling)

for n, p in lm_net.named_parameters():
    if "lora_" in n:
        print(n)
# for n, p in lm_net.named_parameters():
#     if "lora" in n:
#         print(n)

Parameter containing:
tensor(7.8141, requires_grad=True)
Parameter containing:
tensor(7.7674, requires_grad=True)
Parameter containing:
tensor(7.7597, requires_grad=True)
Parameter containing:
tensor(7.8463, requires_grad=True)
Parameter containing:
tensor(7.6999, requires_grad=True)
Parameter containing:
tensor(8.0894, requires_grad=True)
Parameter containing:
tensor(7.7961, requires_grad=True)
Parameter containing:
tensor(7.7531, requires_grad=True)
Parameter containing:
tensor(7.7874, requires_grad=True)
Parameter containing:
tensor(7.6449, requires_grad=True)
Parameter containing:
tensor(7.9886, requires_grad=True)
Parameter containing:
tensor(7.8015, requires_grad=True)
Parameter containing:
tensor(7.6323, requires_grad=True)
Parameter containing:
tensor(7.7711, requires_grad=True)
Parameter containing:
tensor(7.5944, requires_grad=True)
Parameter containing:
tensor(7.8987, requires_grad=True)
Parameter containing:
tensor(7.7331, requires_grad=True)
Parameter containing:
tensor(7.